In [76]:
import pandas as pd, numpy as np, random

In [2]:
# Loads a data file from a provided file location.
def load_data(path):
    # Your code here:
    loaded_data = pd.read_csv(path, parse_dates= ["date"])
    return loaded_data

In [52]:
data = load_data("C://Users/gbeng/OneDrive/Desktop/AI_534/IA1_train.csv")

In [53]:
data.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,7972604355,2014-05-21,3,1.00,1020,7874,1.0,0,0,3,...,1020,0,1956,0,98106,47.5175,-122.346,1290,7320,2.1800
1,8731951130,2014-06-09,3,2.25,2210,8000,2.0,0,0,4,...,2210,0,1969,0,98023,47.3085,-122.381,1990,8000,2.5025
2,7885800740,2015-02-18,4,2.50,2350,5835,2.0,0,0,3,...,2350,0,2003,0,98042,47.3494,-122.153,3010,5772,2.7000
3,4232900940,2014-05-22,3,1.50,1660,4800,2.0,0,0,3,...,1660,0,1907,0,98119,47.6352,-122.358,1690,4000,9.2630
4,3275850190,2014-09-05,3,2.50,2410,9916,2.0,0,0,4,...,2410,0,1989,0,98052,47.6911,-122.103,2310,8212,7.0000


In [54]:
# Implements dataset preprocessing, with boolean options to either normalize the data or not, 
# and to either drop the sqrt_living15 column or not.
#
# Note that you will call this function multiple times to generate dataset versions that are
# / aren't normalized, or versions that have / lack sqrt_living15.
def preprocess_data(data, normalize:bool=False, drop_sqrt_living15:bool=False):
    # Your code here:
    data["year"] = data["date"].dt.year
    data["month"] = data["date"].dt.month
    data["day"] = data["date"].dt.day
    data.insert(0, "bias", 1)
    data.drop(["id", "date"], axis=1, inplace=True)
    
    
    age_since_renovated = np.zeros(len(data))
    
    for (i, value) in enumerate(data["yr_renovated"]):
        if value==0:
            age_since_renovated[i] = data["year"][i] - data["yr_built"][i]
        else:
            age_since_renovated[i] = data["year"][i] - data["yr_renovated"][i]
            
    data["age_since_renovated"] = age_since_renovated
    
    
    if normalize:
        global params
        params = {}
        for col in data.columns:
            if col != "price" and col!= "waterfront" and col != "bias":
                μ = np.mean(data[col])
                σ = np.std(data[col])
                data[col] = (data[col] -μ)/ σ
                params[col] = (μ, σ)
                
    if drop_sqrt_living15:
        data.drop(sqrt_living15, axis=1, inplace=True)
     
    preprocessed_data = data.copy()        

    return preprocessed_data

In [55]:
preprocessed_data = preprocess_data(data, True)

In [56]:
2!=1 and 3

3

In [58]:
data.head()

,bias,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,year,month,day,age_since_renovated
0,1,-0.388544,-1.465705,-1.159182,-0.171149,-0.92239,0,-0.304506,-0.634224,-0.56305,...,0.512311,-0.309008,-0.932838,-1.018567,-0.185655,2.1800,-0.682254,-0.513768,0.600460,0.587840
1,1,-0.388544,0.169169,0.145012,-0.168143,0.91779,0,-0.304506,0.888980,0.29038,...,-1.040699,-1.816746,-1.179707,0.004059,-0.162518,2.5025,-0.682254,-0.193389,-0.793764,0.137865
2,1,0.649997,0.496143,0.298447,-0.219801,0.91779,0,-0.304506,-0.634224,0.29038,...,-0.685190,-1.521691,0.428470,1.494170,-0.238328,2.7000,1.465731,-1.474906,0.251904,-1.004378
3,1,-0.388544,-0.811756,-0.457767,-0.244497,0.91779,0,-0.304506,-0.634224,0.29038,...,0.755553,0.540087,-1.017479,-0.434210,-0.298622,9.2630,-0.682254,-0.513768,0.716646,2.283897
4,1,-0.388544,0.496143,0.364205,-0.122426,0.91779,0,-0.304506,0.888980,1.99724,...,-0.498081,0.943353,0.781140,0.471545,-0.155304,7.0000,-0.682254,0.767749,-1.258505,-0.554403


In [59]:
params

{'bedrooms': (3.374125, 0.9628891339998507),
 'bathrooms': (2.12065625, 0.7645851861211542),
 'sqft_living': (2077.684875, 912.4406009824628),
 'sqft_lot': (15046.922875, 41910.28296276945),
 'floors': (1.50125, 0.5434251903436366),
 'view': (0.229, 0.7520365682597641),
 'condition': (3.416375, 0.6565111266193095),
 'grade': (7.65975, 1.1717422658162258),
 'sqft_above': (1788.39775, 831.3623310837061),
 'sqft_basement': (289.287125, 436.58254366639443),
 'yr_built': (1971.01625, 29.477330034070267),
 'yr_renovated': (80.0765, 391.66864458077055),
 'zipcode': (98078.61975, 53.44463172234946),
 'lat': (47.56033410000018, 0.1386182543974276),
 'long': (-122.21374649999989, 0.14177547473999108),
 'sqft_living15': (1987.221875, 684.5124178175687),
 'sqft_lot15': (12776.2765, 29389.2575588435),
 'year': (2014.317625, 0.4655527460718032),
 'month': (6.603625, 3.1212997067527137),
 'day': (15.831875, 8.606936678306342),
 'age_since_renovated': (41.017, 28.890529780535424)}

In [60]:
preprocessed_data

,bias,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,year,month,day,age_since_renovated
0,1,-0.388544,-1.465705,-1.159182,-0.171149,-0.92239,0,-0.304506,-0.634224,-0.56305,...,0.512311,-0.309008,-0.932838,-1.018567,-0.185655,2.1800,-0.682254,-0.513768,0.600460,0.587840
1,1,-0.388544,0.169169,0.145012,-0.168143,0.91779,0,-0.304506,0.888980,0.29038,...,-1.040699,-1.816746,-1.179707,0.004059,-0.162518,2.5025,-0.682254,-0.193389,-0.793764,0.137865
2,1,0.649997,0.496143,0.298447,-0.219801,0.91779,0,-0.304506,-0.634224,0.29038,...,-0.685190,-1.521691,0.428470,1.494170,-0.238328,2.7000,1.465731,-1.474906,0.251904,-1.004378
3,1,-0.388544,-0.811756,-0.457767,-0.244497,0.91779,0,-0.304506,-0.634224,0.29038,...,0.755553,0.540087,-1.017479,-0.434210,-0.298622,9.2630,-0.682254,-0.513768,0.716646,2.283897
4,1,-0.388544,0.496143,0.364205,-0.122426,0.91779,0,-0.304506,0.888980,1.99724,...,-0.498081,0.943353,0.781140,0.471545,-0.155304,7.0000,-0.682254,0.767749,-1.258505,-0.554403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1,0.649997,-0.484781,-0.085140,-0.183652,-0.92239,0,-0.304506,-0.634224,-0.56305,...,-1.414918,-1.569303,-0.629541,-0.390383,-0.184635,2.6700,1.465731,-1.474906,1.181387,0.380159
7996,1,0.649997,0.169169,-0.227615,-0.199329,0.91779,0,-0.304506,-0.634224,-0.56305,...,-1.414918,-2.041103,-0.601328,-0.492645,-0.212944,2.9900,-0.682254,0.767749,1.181387,-0.796697
7997,1,-1.427085,-1.465705,-0.918071,-0.227794,-0.92239,0,-0.304506,-0.634224,-0.56305,...,0.437467,0.618720,-0.728289,0.135539,-0.285012,9.0000,-0.682254,1.408508,0.368090,0.657067
7998,1,-0.388544,-0.811756,-0.874232,-0.308586,-0.00230,0,-0.304506,-0.634224,0.29038,...,0.811686,0.411677,-0.664808,-0.653344,-0.385184,6.8750,-0.682254,1.088128,-0.212837,2.387737


In [61]:
for col in preprocessed_data:
   print(f"{col}: {np.mean(preprocessed_data[col])}")


bias: 1.0
bedrooms: 7.38978322978312e-16
bathrooms: 1.5583367929394852e-16
sqft_living: 1.242200786677472e-16
sqft_lot: 1.9696397290935863e-16
floors: -3.031325190860912e-16
waterfront: 0.00675
view: 5.260930580064382e-16
condition: 6.06514838352723e-16
grade: 3.559652572704408e-16
sqft_above: -8.778177837398538e-17
sqft_basement: -9.053591210061995e-16
yr_built: 3.350236754684488e-15
yr_renovated: 6.23716356340509e-16
zipcode: 3.687988803235953e-14
lat: -1.3228019513089607e-12
long: -8.083140129455657e-13
sqft_living15: 5.671157987663378e-17
sqft_lot15: -6.091654958240156e-17
price: 5.361672840000017
year: 1.9876847390243313e-13
month: -1.8936241463762826e-16
day: 4.6074255521944e-18
age_since_renovated: -1.0045436704686495e-16


In [90]:
X = preprocessed_data.drop("price", axis=1)
y =preprocessed_data["price"]

In [91]:
X_matrix = np.asmatrix(X.to_numpy())

In [92]:
preprocessed_data.head()

,bias,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,year,month,day,age_since_renovated
0,1,-0.388544,-1.465705,-1.159182,-0.171149,-0.92239,0,-0.304506,-0.634224,-0.56305,...,0.512311,-0.309008,-0.932838,-1.018567,-0.185655,2.1800,-0.682254,-0.513768,0.600460,0.587840
1,1,-0.388544,0.169169,0.145012,-0.168143,0.91779,0,-0.304506,0.888980,0.29038,...,-1.040699,-1.816746,-1.179707,0.004059,-0.162518,2.5025,-0.682254,-0.193389,-0.793764,0.137865
2,1,0.649997,0.496143,0.298447,-0.219801,0.91779,0,-0.304506,-0.634224,0.29038,...,-0.685190,-1.521691,0.428470,1.494170,-0.238328,2.7000,1.465731,-1.474906,0.251904,-1.004378
3,1,-0.388544,-0.811756,-0.457767,-0.244497,0.91779,0,-0.304506,-0.634224,0.29038,...,0.755553,0.540087,-1.017479,-0.434210,-0.298622,9.2630,-0.682254,-0.513768,0.716646,2.283897
4,1,-0.388544,0.496143,0.364205,-0.122426,0.91779,0,-0.304506,0.888980,1.99724,...,-0.498081,0.943353,0.781140,0.471545,-0.155304,7.0000,-0.682254,0.767749,-1.258505,-0.554403


In [93]:
X_matrix[0]

matrix([[ 1.        , -0.38854421, -1.4657049 , -1.15918217, -0.17114947,
         -0.92239007,  0.        , -0.30450647, -0.63422383, -0.56305044,
         -0.92426337, -0.66261725, -0.5094169 , -0.20444961,  0.51231057,
         -0.30900764, -0.93283765, -1.01856717, -0.18565547, -0.68225352,
         -0.51376835,  0.60046044,  0.58783969]])

In [107]:
w = np.random.rand(1, 23)

In [108]:
w_matrix = np.asmatrix(w)

In [109]:
w_matrix.shape

(1, 23)

In [106]:
X.shape

(8000, 23)

In [111]:
ŷ_matrix = w_matrix * np.transpose(X_matrix) 

In [122]:
p = np.transpose(ŷ_matrix)

In [125]:
ŷ = np.squeeze(np.asarray(p))

In [119]:
np.asmatrix(y.to_numpy()).shape

(1, 8000)

In [126]:
K = y - ŷ

In [127]:
K.shape

(8000,)

In [177]:
K.head()

0     7.537478
1     5.247820
2     1.501003
3    11.948091
4     4.564397
Name: price, dtype: float64

In [176]:
K**2

0        56.813575
1        27.539612
2         2.253010
3       142.756867
4        20.833722
           ...    
7995     22.866673
7996     41.107194
7997    161.977243
7998     96.643461
7999     50.521593
Name: price, Length: 8000, dtype: float64

In [223]:
np.ones(23) * np.transpose(X_matrix)

matrix([[-8.85097778, -3.60536012,  2.10348968, ..., -5.6369599 ,
         -3.68504192, -3.8662625 ]])

In [225]:
def gd_train(data, lr):
    X = data.drop("price", axis=1)
    y =data["price"]
    X_matrix = np.asmatrix(X.to_numpy())
    w = np.ones(23)
    weights = {}
    losses = {}
    
    for i in range(0, 4000):
        ŷ_matrix = w * np.transpose(X_matrix) 
        p = np.transpose(ŷ_matrix)
        ŷ = np.squeeze(np.asarray(p))
        diff = np.asmatrix(ŷ - y)
        losses[f"{i}"] = (1/len(X)) * sum((ŷ - y)**2)
        δL = (2/len(X)) * sum(np.squeeze(np.asarray(np.dot(diff ,X_matrix))))
        w = w - lr * δL
        weights[f"{i}"] = w
        
        if abs(δL)<=0.001:
            return weights
    
    
    return weights

In [212]:
o={}
o[1]=2

In [213]:
o

{1: 2}

In [198]:
X_matrix.shape

(8000, 23)

In [205]:
(2/len(X)) * sum(np.squeeze(np.asarray(np.asmatrix(ŷ - y) * X_matrix)))

9.408115076778266

In [199]:
L = np.asmatrix(K)

In [200]:
np.squeeze(np.asarray(p))

array([-5.35747805, -2.74531979,  1.19899694, ..., -3.72702803,
       -2.95574063, -2.90785429])

In [201]:
L-10

matrix([[-2.46252195, -4.75218021, -8.49899694, ...,  2.72702803,
         -0.16925937, -2.89214571]])

In [202]:
ŷ

array([-5.35747805, -2.74531979,  1.19899694, ..., -3.72702803,
       -2.95574063, -2.90785429])

In [203]:
L - ŷ

matrix([[12.89495611,  7.99313957,  0.30200612, ..., 16.45405606,
         12.78648125, 10.01570859]])

In [226]:
gd_train(data, 10)

C:\Users\gbeng\AppData\Local\Temp/ipykernel_3160/2546722240.py:15: RuntimeWarning: invalid value encountered in double_scalars
  δL = (2/len(X)) * sum(np.squeeze(np.asarray(np.dot(diff ,X_matrix))))


{'0': array([-541.08961705, -541.08961705, -541.08961705, -541.08961705,
        -541.08961705, -541.08961705, -541.08961705, -541.08961705,
        -541.08961705, -541.08961705, -541.08961705, -541.08961705,
        -541.08961705, -541.08961705, -541.08961705, -541.08961705,
        -541.08961705, -541.08961705, -541.08961705, -541.08961705,
        -541.08961705, -541.08961705, -541.08961705]),
 '1': array([535944.82425711, 535944.82425711, 535944.82425711, 535944.82425711,
        535944.82425711, 535944.82425711, 535944.82425711, 535944.82425711,
        535944.82425711, 535944.82425711, 535944.82425711, 535944.82425711,
        535944.82425711, 535944.82425711, 535944.82425711, 535944.82425711,
        535944.82425711, 535944.82425711, 535944.82425711, 535944.82425711,
        535944.82425711, 535944.82425711, 535944.82425711]),
 '2': array([-5.30404193e+08, -5.30404193e+08, -5.30404193e+08, -5.30404193e+08,
        -5.30404193e+08, -5.30404193e+08, -5.30404193e+08, -5.30404193e+0